In [6]:
!pip install numpy pandas geopandas torch node2vec

  Using cached node2vec-0.5.0-py3-none-any.whl.metadata (849 bytes)
  Using cached gensim-4.3.3-cp312-cp312-win_amd64.whl.metadata (8.2 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     --------------------- ------------------ 30.7/57.7 kB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 57.7/57.7 kB 1.5 MB/s eta 0:00:00
  Using cached scipy-1.13.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached smart_open-7.0.5-py3-none-any.whl.metadata (24 kB)
Using cached node2vec-0.5.0-py3-none-any.whl (7.2 kB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
Using cached gensim-4.3.3-cp312-cp312-win_amd64.whl (24.0 MB)
   ---------------------------------------- 0.0/301.8 kB ? eta -:--:--
   --------------------------------------- 301.8/301.8 kB 19.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/78.5 kB ? eta -:--:--
   ------------------

  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip cache purge
!pip install numpy==1.22.4 gensim==4.3.3

In [ ]:
import math
import os
import sys
import logging

import numpy as np
import pandas as pd
import geopandas as gpd
import torch
import torch.nn as nn
import networkx as nx

from torch.utils.data import DataLoader, Dataset
from gensim.models import KeyedVectors
from node2vec import Node2Vec
from shapely.geometry import Point, MultiPolygon
from shapely.wkt import loads

ModuleNotFoundError: No module named 'numpy.rec'

In [ ]:
logger = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

**Load Dataset Features**

In [ ]:
try:
  from google.colab import drive
  logger.info("Running on Google Colab, reading dataset from drive")
  drive.mount("/content/drive")
  DATASET_PATH = "/content/drive/MyDrive/ECE2500/EdmontonFireRescueServicesData"
except:
  logger.info("Running locally, reading dataset from local file system")
  DATASET_PATH = "./dataset/EdmontonFireRescueServicesData"
  if not os.path.exists(DATASET_PATH):
    logger.critical(f"Cannot find dataset directory, place dataset in {DATASET_PATH}")
    exit(1)

UNIT_TRIP_PATH = os.path.join(DATASET_PATH, "EFRS_Unit_Trip_Summary.csv")
WEEKLY_EVENTS_PATH = os.path.join(DATASET_PATH, "weekly_events.csv")
NEIGHBOURHOOD_PATH = os.path.join(DATASET_PATH, "City_of_Edmonton_-_Neighbourhoods_20241022.csv")
NEIGHBOURHOOD_FEATURES_PATH = os.path.join(DATASET_PATH, "neighbourhood_features.csv")

logger.debug(f"Unit Trip: {UNIT_TRIP_PATH}")
logger.debug(f"Weekly Events: {WEEKLY_EVENTS_PATH}")
logger.debug(f"Neighbourhood: {NEIGHBOURHOOD_PATH}")
logger.debug(f"Neighbourhood Features: {NEIGHBOURHOOD_FEATURES_PATH}")

unit_trip_df = pd.read_csv(UNIT_TRIP_PATH)
weekly_events_df = pd.read_csv(WEEKLY_EVENTS_PATH)
neighbourhood_df = pd.read_csv(NEIGHBOURHOOD_PATH)
neighbourhood_feature_df = pd.read_csv(NEIGHBOURHOOD_FEATURES_PATH)

INFO:__main__:Running locally, reading dataset from local file system


C:\Users\hardy\AppData\Local\Temp\ipykernel_21392\998725363.py:23: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  unit_trip_df = pd.read_csv(UNIT_TRIP_PATH)


**Data and Embedding**

In [ ]:
def generate_node2vec_embeddings(neighbourhood_info_df, node2vec_dim=32):
  """
  Generates Node2Vec embeddings for neighborhoods based on a sample adjacency graph.
  node2vec_dim: Dimension of the embeddings to be generated.
  """

  num_neighborhood_info = len(neighbourhood_info_df)
  G = nx.Graph()

  # Convert "Geometry Multipolygon" column to GeoSeries
  neighbourhood_info_df['geometry'] = gpd.GeoSeries.from_wkt(neighbourhood_info_df['Geometry Multipolygon'])
  neighbourhood_info_df['nid'] = range(num_neighborhood_info)

  # Assuming you have a way to define neighborhood connections based on proximity
  # You can use the geometry information for this.
  # Here's a placeholder for how you might connect neighborhoods based on proximity:

  for i in range(num_neighborhood_info):
    for j in range(i + 1, num_neighborhood_info):
      # Use the new 'geometry' column for spatial operations
      if neighbourhood_info_df['geometry'].iloc[i].intersects(neighbourhood_info_df['geometry'].iloc[j]):
        G.add_edge(i, j)

  # Alternatively, you could build a graph based on other criteria like sharing a boundary
  node2vec = Node2Vec(G, dimensions=node2vec_dim, walk_length=10, num_walks=100, p=1, q=1)
  node2vec_model = node2vec.fit()
  node2vec_embeddings_np = np.array([node2vec_model.wv[str(i)] for i in range(num_neighborhood_info)])
  node2vec_embeddings = torch.from_numpy(node2vec_embeddings_np)
  node2vec_emb_layer = nn.Embedding.from_pretrained(node2vec_embeddings, freeze=True)

  return node2vec_emb_layer


class Time2Vec(nn.Module):
  """
  Time2Vec embedding module for temporal features

  This captures both linear and periodic components for time-based features.
  """
  def __init__(self, input_dim, embed_dim, act_function=torch.sin):
    super(Time2Vec, self).__init__()
    self.embed_dim = embed_dim // input_dim  # Embedding dimension per time feature
    self.act_function = act_function       # Activation function for periodicity
    self.weight = nn.Parameter(torch.randn(input_dim, self.embed_dim))
    self.bias = nn.Parameter(torch.randn(input_dim, self.embed_dim))

  def forward(self, x):
    # Diagonal embedding for each time feature (day of week, hour, etc.)
    x = torch.diag_embed(x)
    x_affine = torch.matmul(x, self.weight) + self.bias
    x_affine_0, x_affine_remain = torch.split(x_affine, [1, self.embed_dim - 1], dim=-1)
    x_affine_remain = self.act_function(x_affine_remain)
    return torch.cat([x_affine_0, x_affine_remain], dim=-1).view(x.size(0), x.size(1), -1)


class NeighborhoodDataset(Dataset):
    def __init__(self, neighborhood_ids, time_features, building_type_ids, building_counts,
                 population, event_type_ids, equipment_ids, targets):
        self.neighborhood_ids = neighborhood_ids  # Tensor of input neighborhood_ids
        self.time_features = time_features  # Tensor of input time_features
        self.building_type_ids = building_type_ids  # Tensor of input building_type_ids
        self.building_counts = building_counts  # Tensor of input building_counts
        self.population = population  # Tensor of input population
        self.event_type_ids = event_type_ids  # Tensor of input event_type_ids
        self.equipment_ids = equipment_ids  # Tensor of input equipment_ids
        self.targets = targets    # Tensor of target values

    def __len__(self):
        return len(self.neighborhood_ids)  # Number of neighborhoods

    def __getitem__(self, idx):
        return (self.neighborhood_ids[idx], self.time_features[idx], self.building_type_ids[idx], self.building_counts[idx],
                self.population[idx], self.event_type_ids[idx], self.equipment_ids[idx], self.targets[idx])


class CombinedEmbedding(nn.Module):
  """
  Combined Embedding Module

  Combines embeddings from Node2Vec, Time2Vec, building type/counts, population, event type, and equipment.
  Projects the combined embedding to a target dimension (e.g., 64) for compatibility with transformer layers.
  """
  def __init__(self, node2vec_emb_layer, time2vec_embed_dim, time_feature_dim,
          num_building_types, building_type_embed_dim, population_embed_dim,
          num_event_types, event_type_embed_dim, num_equipment_types, equipment_embed_dim,
          target_embed_dim=64):  # Add target_embed_dim for projection
    super(CombinedEmbedding, self).__init__()

    # Embedding initialization code
    self.node2vec_emb_layer = node2vec_emb_layer  # Precomputed Node2Vec embeddings
    self.time2vec = Time2Vec(input_dim=time_feature_dim, embed_dim=time2vec_embed_dim)
    self.building_type_embedding = nn.Embedding(num_building_types, building_type_embed_dim)
    self.population_embedding = nn.Linear(1, population_embed_dim)
    self.event_type_embedding = nn.Embedding(num_event_types, event_type_embed_dim)
    self.equipment_embedding = nn.Embedding(num_equipment_types, equipment_embed_dim)

    # Compute the combined embedding dimension before projection
    # num_neighbourhood * 2 (month + year) * x
    self.projection_dim = (node2vec_emb_layer.embedding_dim + time2vec_embed_dim +
                    building_type_embed_dim + population_embed_dim +
                    event_type_embed_dim + equipment_embed_dim)

    # Projection layer to reduce to target_embed_dim
    self.projection_layer = nn.Linear(self.projection_dim, target_embed_dim)
    
  def forward(self, neighborhood_ids, time_features, building_type_ids, building_counts,
            population, event_type_ids, equipment_ids):
    # Generate embeddings
    spatial_embeddings = self.node2vec_emb_layer(neighborhood_ids).unsqueeze(1).repeat(1, time_features.size(1), 1)
    logger.debug(f"Spatial Embedding Shape: {spatial_embeddings.shape}")

    temporal_embeddings = self.time2vec(time_features)
    logger.debug(f"Temporal Embedding Shape: {temporal_embeddings.shape}")

    #building_embeddings = (self.building_type_embedding(building_type_ids) * building_counts).sum(dim=2)
    #building_embeddings = building_embeddings.unsqueeze(1).repeat(1, time_features.size(1), 1)
    #logger.debug(f"Building Embedding Shape: {building_embeddings.shape}")
    
    
    # Building embeddings
    building_type_embeds = self.building_type_embedding(building_type_ids)  # Shape: [batch_size, num_building_types, building_type_embed_dim]
    logger.debug(f"Building Type Embeds Shape: {building_type_embeds.shape}")

    # Adjust building_counts to match building_type_embed_dim
    building_counts = building_counts.unsqueeze(-1)  # Shape: [batch_size, 1, num_building_types, 1]
    building_counts = building_counts.repeat(1, 1, 1, self.building_type_embedding.embedding_dim)  # Match embed_dim
    logger.debug(f"Building Counts Shape after adjustment: {building_counts.shape}")

    # Multiply and aggregate
    building_embeddings = (building_type_embeds.unsqueeze(1) * building_counts).sum(dim=2)  # Shape: [batch_size, 1, building_type_embed_dim]
    building_embeddings = building_embeddings.repeat(1, time_features.size(1), 1)  # Match temporal dimension
    logger.debug(f"Building Embedding Shape: {building_embeddings.shape}")

    population_embeddings = self.population_embedding(population.unsqueeze(-1)).unsqueeze(1).repeat(1, time_features.size(1), 1)
    logger.debug(f"Population Embedding Shape: {population_embeddings.shape}")

    event_type_embeddings = self.event_type_embedding(event_type_ids)
    logger.debug(f"Event Type Embedding Shape: {event_type_embeddings.shape}")

    equipment_embeddings = self.equipment_embedding(equipment_ids)
    logger.debug(f"Equipment Embedding Shape: {equipment_embeddings.shape}")

    # Concatenate all embeddings
    combined_embedding = torch.cat([spatial_embeddings, temporal_embeddings, building_embeddings,
                                     population_embeddings, event_type_embeddings, equipment_embeddings], dim=-1)
    logger.debug(f"Combined Embedding Shape before Projection: {combined_embedding.shape}")

    # Project to target dimension
    combined_embedding = self.projection_layer(combined_embedding)
    return combined_embedding

  """
  def forward(self, neighborhood_ids, time_features, building_type_ids, building_counts,
              population, event_type_ids, equipment_ids):
    # Generate and combine embeddings
    temporal_embeddings = self.time2vec(time_features)
    logger.debug(f"Temporal Embedding Shape: {temporal_embeddings.shape}")
    # Albert: spatial_embeddings = self.node2vec_emb_layer(neighborhood_ids).unsqueeze(1).repeat(1, temporal_embeddings.size(1), 1) # [batch_size, temporal_dimensions, time2vec_embed_dim]
    
    logger.debug(f"Spatial Embedding Shape: {spatial_embeddings.shape}")
    building_embeddings = (self.building_type_embedding(building_type_ids) * building_counts.unsqueeze(-1)).sum(dim=2)
    logger.debug(f"Building Embedding Shape: {building_embeddings.shape}")
    population_embeddings = self.population_embedding(population.unsqueeze(-1)).unsqueeze(1).expand(-1, spatial_embeddings.size(1), -1)
    logger.debug(f"Population Embedding Shape: {population_embeddings.shape}")
    event_type_embeddings = self.event_type_embedding(event_type_ids)
    logger.debug(f"Event Type Embedding Shape: {event_type_embeddings.shape}")
    equipment_embeddings = self.equipment_embedding(equipment_ids)
    logger.debug(f"Equipment Embedding Shape: {equipment_embeddings.shape}")

    # Concatenate all embeddings into a single combined embedding
    combined_embedding = torch.cat([spatial_embeddings, temporal_embeddings,
                                    building_embeddings, population_embeddings,
                                    event_type_embeddings, equipment_embeddings], dim=-1)

    logger.debug(f"Combined Embedding Shape before Projection: {combined_embedding.shape}")

    # Project to target dimension for compatibility (e.g., 64)
    combined_embedding = self.projection_layer(combined_embedding)

    return combined_embedding
  """

class PositionalEncoding(nn.Module):
  """
  Positional Encoding Module
  """
  def __init__(self, embed_dim, max_len=7):  # 7 days in a week
    super(PositionalEncoding, self).__init__()
    position = torch.arange(0, max_len).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, embed_dim, 2) * -(math.log(10000.0) / embed_dim))
    pe = torch.zeros(max_len, embed_dim)
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    self.pe = pe.unsqueeze(0)  # Shape: (1, max_len, embed_dim)

  def forward(self, x):
    x = x + self.pe[:, :x.size(1), :].to(x.device)
    return x

**Transformer Model**

In [ ]:
# Transformer-based Emergency Event Predictor
class EmergencyEventPredictor(nn.Module):
  def __init__(self, embedding_module, embed_dim, num_heads, num_layers, max_len=7):
    super(EmergencyEventPredictor, self).__init__()

    # Embedding module (CombinedEmbedding) and positional encoding
    self.embedding_module = embedding_module
    self.positional_encoding = PositionalEncoding(embed_dim, max_len)

    # Transformer Encoder
    encoder_layer = nn.TransformerEncoderLayer(
        d_model=embed_dim, nhead=num_heads, dim_feedforward=512, dropout=0.1
    )
    self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    # Prediction head
    self.fc_out = nn.Linear(embed_dim, 1)  # Output: predicting the number of events

  def forward(self, neighborhood_ids, time_features, building_type_ids, building_counts,
              population, event_type_ids, equipment_ids):

    # Generate combined embeddings from the embedding module
    x = self.embedding_module(neighborhood_ids, time_features, building_type_ids, building_counts,
                              population, event_type_ids, equipment_ids)

    # Apply positional encoding
    x = self.positional_encoding(x)

    # Pass through transformer encoder
    x = self.transformer_encoder(x)

    # Prediction layer (we apply it to each element in the sequence)
    # Albert: predictions = self.fc_out(x).squeeze(-1)  # Shape: [batch_size]
    predictions = self.fc_out(x.squeeze(1)).squeeze(-1)  # Shape: [batch_size]

    return predictions

**Feature Extraction**

In [ ]:
build_type_list = [
    'Apartment_Condo_1_to_4_stories', 'Apartment_Condo_5_or_more_stories',
    'Duplex_Fourplex', 'Hotel_Motel',
    'Institution_Collective_Residence', 'Manufactured_Mobile_Home',
    'RV_Tent_Other', 'Row_House',
    'Single_Detached_House'
    ]
event_type_list = weekly_events_df['Rc_description'].unique()
unit_type_list = unit_trip_df['unityp'].unique()

num_neighborhoods = len(neighbourhood_feature_df)
num_building_types = len(build_type_list)
num_event_types = len(event_type_list)
num_equipment_types = len(unit_type_list)

neighbourhood_mappings = weekly_events_df["Neighbourhood Number"].unique()
building_counts_np = neighbourhood_feature_df[build_type_list].fillna(0).astype(int).to_numpy()
population_np = neighbourhood_feature_df['Population'].fillna(0).astype(int).to_numpy()

**Transformer Model Initalization**

In [ ]:
# Embedding Parameter and Module

node2vec_dim = 32
time2vec_embed_dim = 64
time_feature_dim = 2  # week, year
building_type_embed_dim = 16
population_embed_dim = 8
event_type_embed_dim = 16
equipment_embed_dim = 16
target_embed_dim = 64

node2vec_emb_layer = generate_node2vec_embeddings(
    neighbourhood_info_df=neighbourhood_df,
    node2vec_dim=node2vec_dim
)

embedding_module = CombinedEmbedding(
    node2vec_emb_layer=node2vec_emb_layer,
    time2vec_embed_dim=time2vec_embed_dim,
    time_feature_dim=time_feature_dim,
    num_building_types=num_building_types,
    building_type_embed_dim=building_type_embed_dim,
    population_embed_dim=population_embed_dim,
    num_event_types=num_event_types,
    event_type_embed_dim=event_type_embed_dim,
    num_equipment_types=num_equipment_types,
    equipment_embed_dim=equipment_embed_dim,
    target_embed_dim=target_embed_dim
)

Computing transition probabilities:   0%|          | 0/403 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 100/100 [00:01<00:00, 82.20it/s]

INFO:gensim.models.word2vec:collecting all words and their counts


INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 100000 words, keeping 403 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #20000, processed 200000 words, keeping 403 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #30000, processed 300000 words, keeping 403 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #40000, processed 400000 words, keeping 403 word types
INFO:gensim.models.word2vec:collected 403 word types from a corpus of 403000 raw words and 40300 sentences
INFO:gensim.models.word2vec:Creating a fresh vocabulary
INFO:gensim.utils:Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 403 unique words (100.00% of original 403, drops 0)', 'datetime': '2024-11-25T02:15:00.397766', 'gensim': '4.3.3', 'python': '3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 15:03:56) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Wind

In [ ]:
# Define batch and mini-batch

mini_batch_size = 29
batch_size = 13

spatial_dimension = num_neighborhoods

# verify spatial_dimension = batch_size * mini_batch_size
assert spatial_dimension == batch_size * mini_batch_size

In [ ]:
# Neighborhood Features
neighborhood_ids = torch.arange(num_neighborhoods)
logger.info(f"neighborhood_ids shape {neighborhood_ids.shape}")

# Time Feature
time_features = torch.zeros(spatial_dimension, 1, time_feature_dim)
for nid in range(num_neighborhoods):
    neighborhood_data = weekly_events_df[weekly_events_df["Neighbourhood Number"] == neighbourhood_mappings[nid]]
    if not neighborhood_data.empty:
        year_mean = neighborhood_data["year"].mean()
        week_mean = neighborhood_data["week_of_year"].mean()
        time_features[nid, 0, :] = torch.tensor([year_mean, week_mean])
    else:
        time_features[nid, 0, :] = torch.zeros(time_feature_dim)
logger.info(f"time_features shape: {time_features.shape}")

# Building Features
building_type_ids = torch.arange(num_building_types).repeat(spatial_dimension, 1)
logger.info(f"building_type_ids shape {building_type_ids.shape}")

building_counts_np = neighbourhood_feature_df[build_type_list].fillna(0).to_numpy(dtype=np.int32)
temporal_dimension = 1
building_counts = torch.from_numpy(building_counts_np).unsqueeze(1).repeat(1, temporal_dimension, 1)
logger.info(f"building_counts shape {building_counts.shape}")

# Demographic Features
population = torch.from_numpy(population_np).float()
logger.info(f"population shape {population.shape}")

# Event Features
event_type_ids = torch.randint(0, num_event_types, (spatial_dimension,))
logger.info(f"event_type_ids shape {event_type_ids.shape}")

equipment_ids = torch.randint(0, num_equipment_types, (spatial_dimension,))
logger.info(f"equipment_ids shape {equipment_ids.shape}")

# Target Values
agg_data = weekly_events_df.groupby(['year', 'week_of_year', 'Neighbourhood Number']).agg({
    'Rc_description': ' '.join,
    'event_count': 'sum'
}).reset_index()

unique_week_year_combinations = agg_data[['year', 'week_of_year']].drop_duplicates()
num_combinations = len(unique_week_year_combinations)

targets = torch.zeros((num_neighborhoods, num_combinations), dtype=torch.float32)
for i, neighbourhood in enumerate(neighbourhood_feature_df['Neighbourhood_Number']):
    neighbourhood_data = agg_data[agg_data['Neighbourhood Number'] == neighbourhood]
    for k, (year, week) in enumerate(unique_week_year_combinations.itertuples(index=False)):
        event_count = neighbourhood_data[(neighbourhood_data['year'] == year) &
                                         (neighbourhood_data['week_of_year'] == week)]['event_count'].sum() or 0
        targets[i, k] = event_count
logger.info(f"targets shape {targets.shape}")


INFO:__main__:neighborhood_ids shape torch.Size([377])
INFO:__main__:time_features shape: torch.Size([377, 1, 2])
INFO:__main__:building_type_ids shape torch.Size([377, 9])
INFO:__main__:building_counts shape torch.Size([377, 1, 9])
INFO:__main__:population shape torch.Size([377])
INFO:__main__:event_type_ids shape torch.Size([377])
INFO:__main__:equipment_ids shape torch.Size([377])
INFO:__main__:targets shape torch.Size([377, 623])


**Transformer Model Training Loop**

In [ ]:
# Neighborhood Features
neighborhood_ids = torch.arange(num_neighborhoods)
logger.info(f"neighborhood_ids shape {neighborhood_ids.shape}")

# Time Features
time_features = torch.zeros(spatial_dimension, 1, time_feature_dim)
for nid in range(num_neighborhoods):
    # Filter rows for the current neighborhood
    neighborhood_data = weekly_events_df[weekly_events_df["Neighbourhood Number"] == neighbourhood_mappings[nid]]

    if not neighborhood_data.empty:
        # Use "year" and "week_of_year" as features
        year_mean = neighborhood_data["year"].mean()
        week_mean = neighborhood_data["week_of_year"].mean()

        # Combine them into time features (e.g., [mean year, mean week])
        time_features[nid, 0, :] = torch.tensor([year_mean, week_mean])
    else:
        # Default to zero if no data for the neighborhood
        time_features[nid, 0, :] = torch.zeros(time_feature_dim)

logger.info(f"time_features shape: {time_features.shape}")

# Building Features
building_type_ids = torch.arange(num_building_types).repeat(spatial_dimension, 1)
logger.info(f"building_type_ids shape {building_type_ids.shape}")

# Building Counts
building_counts_np = neighbourhood_feature_df[build_type_list].fillna(0).to_numpy(dtype=np.int32)
building_counts = torch.from_numpy(building_counts_np).unsqueeze(1)  # Temporal dimension = 1
logger.info(f"building_counts shape {building_counts.shape}")

# Demographic Features
population = torch.from_numpy(population_np).float()
logger.info(f"population shape {population.shape}")

# Event Features
event_type_ids = torch.randint(0, num_event_types, (spatial_dimension, 1))  # Temporal dimension = 1
logger.info(f"event_type_ids shape {event_type_ids.shape}")

equipment_ids = torch.randint(0, num_equipment_types, (spatial_dimension, 1))  # Temporal dimension = 1
logger.info(f"equipment_ids shape {equipment_ids.shape}")

# Target Values
agg_data = weekly_events_df.groupby(['year', 'week_of_year', 'Neighbourhood Number']).agg({
    'Rc_description': ' '.join,  # Concatenate descriptions (or other aggregation if needed)
    'event_count': 'sum'         # Sum event counts
}).reset_index()

unique_week_year_combinations = agg_data[['year', 'week_of_year']].drop_duplicates()
targets = torch.zeros((num_neighborhoods,), dtype=torch.float32)

for i, neighbourhood in enumerate(neighbourhood_feature_df['Neighbourhood_Number']):
    neighbourhood_data = agg_data[agg_data['Neighbourhood Number'] == neighbourhood]
    # Sum of event counts for each (year, week_of_year) combination
    for k, (year, week) in enumerate(unique_week_year_combinations.itertuples(index=False)):
        event_count = neighbourhood_data[(neighbourhood_data['year'] == year) &
                                         (neighbourhood_data['week_of_year'] == week)]['event_count'].sum()
        targets[i] = event_count  # Aggregated for the entire week
logger.info(f"targets shape {targets.shape}")

# Instantiate the DataLoader and EmergencyEventPredictor
dataset = NeighborhoodDataset(neighborhood_ids, time_features, building_type_ids, building_counts,
                              population, event_type_ids, equipment_ids, targets)
dataloader = DataLoader(dataset, batch_size=mini_batch_size, shuffle=True)

model = EmergencyEventPredictor(
    embedding_module=embedding_module,
    embed_dim=64,
    num_heads=4,
    num_layers=2
)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.PoissonNLLLoss()

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for i, (_neighborhood_ids, _time_features, _building_type_ids, _building_counts,
            _population, _event_type_ids, _equipment_ids, _targets) in enumerate(dataloader):
        optimizer.zero_grad()

        # Forward pass
        _predictions = model(_neighborhood_ids, _time_features, _building_type_ids,
                             _building_counts, _population, _event_type_ids, _equipment_ids)

        # Compute loss
        loss = criterion(_predictions, _targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        mini_batch_loss = loss.item()
        epoch_loss += mini_batch_loss
        logger.info(f"Epoch [{epoch+1}/{num_epochs}], Mini-batch [{i + 1}/{len(dataloader)}], Loss: {mini_batch_loss:.4f}")

    logger.info(f"Epoch [{epoch+1}/{num_epochs}] completed, Average Loss: {(epoch_loss / len(dataloader)):.4f}")

# Save the trained model
torch.save(model.state_dict(), "transformer_model_weekly.pth")
logger.info("Model saved successfully.")


INFO:__main__:neighborhood_ids shape torch.Size([377])
INFO:__main__:time_features shape: torch.Size([377, 1, 2])
INFO:__main__:building_type_ids shape torch.Size([377, 9])
INFO:__main__:building_counts shape torch.Size([377, 1, 9])
INFO:__main__:population shape torch.Size([377])
INFO:__main__:event_type_ids shape torch.Size([377, 1])
INFO:__main__:equipment_ids shape torch.Size([377, 1])
INFO:__main__:targets shape torch.Size([377])


c:\Users\hardy\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


INFO:__main__:Epoch [1/10], Mini-batch [1/13], Loss: 1.3755
INFO:__main__:Epoch [1/10], Mini-batch [2/13], Loss: 1.2624
INFO:__main__:Epoch [1/10], Mini-batch [3/13], Loss: 0.7101
INFO:__main__:Epoch [1/10], Mini-batch [4/13], Loss: 1.2983
INFO:__main__:Epoch [1/10], Mini-batch [5/13], Loss: 0.6656
INFO:__main__:Epoch [1/10], Mini-batch [6/13], Loss: 2.6776
INFO:__main__:Epoch [1/10], Mini-batch [7/13], Loss: 0.9738
INFO:__main__:Epoch [1/10], Mini-batch [8/13], Loss: 0.7871
INFO:__main__:Epoch [1/10], Mini-batch [9/13], Loss: 0.7652
INFO:__main__:Epoch [1/10], Mini-batch [10/13], Loss: 0.9051
INFO:__main__:Epoch [1/10], Mini-batch [11/13], Loss: 0.8812
INFO:__main__:Epoch [1/10], Mini-batch [12/13], Loss: 0.7460
INFO:__main__:Epoch [1/10], Mini-batch [13/13], Loss: 0.8881
INFO:__main__:Epoch [1/10] completed, Average Loss: 1.0720
INFO:__main__:Epoch [2/10], Mini-batch [1/13], Loss: 1.0434
INFO:__main__:Epoch [2/10], Mini-batch [2/13], Loss: 0.6806
INFO:__main__:Epoch [2/10], Mini-batc

In [ ]:
# Instantiate the DataLoader and EmergencyEventPredictor
dataset = NeighborhoodDataset(neighborhood_ids, time_features, building_type_ids, building_counts,
                              population, event_type_ids, equipment_ids, targets)
dataloader = DataLoader(dataset, batch_size=mini_batch_size, shuffle=True)

model = EmergencyEventPredictor(
    embedding_module=embedding_module,
    embed_dim=64,
    num_heads=4,
    num_layers=2
)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.PoissonNLLLoss()

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
  model.train()
  epoch_loss = 0

  print(f"neighborhood_ids 1 size {neighborhood_ids.shape}")

  for i, (_neighborhood_ids, _time_features, _building_type_ids, _building_counts,
          _population, _event_type_ids, _equipment_ids, _targets) in enumerate(dataloader):
    optimizer.zero_grad()

    # Forward pass
    _predictions = model(_neighborhood_ids, _time_features, _building_type_ids,
                        _building_counts, _population, _event_type_ids, _equipment_ids)

    # Compute loss
    loss = criterion(_predictions, _targets)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    mini_batch_loss = loss.item()
    epoch_loss += mini_batch_loss
    logger.info(f"Epoch [{epoch+1}/{num_epochs}], Mini-batch [{i + 1}/{len(dataloader)}], Loss: {mini_batch_loss:.4f}")

  logger.info(f"Epoch [{epoch+1}/{num_epochs}] completed, Average Loss: {(epoch_loss / len(dataloader)):.4f}")

print(f"neighborhood_ids 2 size {neighborhood_ids.shape}")
# # Save the trained model
# torch.save(model.state_dict(), "transformer_model.pth")

neighborhood_ids 1 size torch.Size([377])
INFO:__main__:Epoch [1/10], Mini-batch [1/13], Loss: 0.9529
INFO:__main__:Epoch [1/10], Mini-batch [2/13], Loss: 0.8118
INFO:__main__:Epoch [1/10], Mini-batch [3/13], Loss: 0.8702
INFO:__main__:Epoch [1/10], Mini-batch [4/13], Loss: 0.7552
INFO:__main__:Epoch [1/10], Mini-batch [5/13], Loss: 0.9175
INFO:__main__:Epoch [1/10], Mini-batch [6/13], Loss: 1.7678
INFO:__main__:Epoch [1/10], Mini-batch [7/13], Loss: 0.8040
INFO:__main__:Epoch [1/10], Mini-batch [8/13], Loss: 0.6451
INFO:__main__:Epoch [1/10], Mini-batch [9/13], Loss: 0.7076
INFO:__main__:Epoch [1/10], Mini-batch [10/13], Loss: 0.6906
INFO:__main__:Epoch [1/10], Mini-batch [11/13], Loss: 1.0096
INFO:__main__:Epoch [1/10], Mini-batch [12/13], Loss: 0.5660
INFO:__main__:Epoch [1/10], Mini-batch [13/13], Loss: 1.0300
INFO:__main__:Epoch [1/10] completed, Average Loss: 0.8868
neighborhood_ids 1 size torch.Size([377])
INFO:__main__:Epoch [2/10], Mini-batch [1/13], Loss: 0.6182
INFO:__main__

c:\Users\hardy\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


INFO:__main__:Epoch [2/10], Mini-batch [3/13], Loss: 0.8366
INFO:__main__:Epoch [2/10], Mini-batch [4/13], Loss: 0.5476
INFO:__main__:Epoch [2/10], Mini-batch [5/13], Loss: 0.9362
INFO:__main__:Epoch [2/10], Mini-batch [6/13], Loss: 0.7679
INFO:__main__:Epoch [2/10], Mini-batch [7/13], Loss: 0.7617
INFO:__main__:Epoch [2/10], Mini-batch [8/13], Loss: 0.7736
INFO:__main__:Epoch [2/10], Mini-batch [9/13], Loss: -0.4613
INFO:__main__:Epoch [2/10], Mini-batch [10/13], Loss: 1.0536
INFO:__main__:Epoch [2/10], Mini-batch [11/13], Loss: 0.9445
INFO:__main__:Epoch [2/10], Mini-batch [12/13], Loss: 1.0618
INFO:__main__:Epoch [2/10], Mini-batch [13/13], Loss: 0.7591
INFO:__main__:Epoch [2/10] completed, Average Loss: 0.7377
neighborhood_ids 1 size torch.Size([377])
INFO:__main__:Epoch [3/10], Mini-batch [1/13], Loss: 0.7865
INFO:__main__:Epoch [3/10], Mini-batch [2/13], Loss: 0.8658
INFO:__main__:Epoch [3/10], Mini-batch [3/13], Loss: 0.7609
INFO:__main__:Epoch [3/10], Mini-batch [4/13], Loss: 0

**Transformer Model Prediction**

In [ ]:
# Define model parameters
embed_dim = target_embed_dim  # Same as output dimension of CombinedEmbedding
num_heads = 4
num_layers = 2
max_len = 7  # Sequence length (e.g., 7 days for a weekly prediction)

# Instantiate the model
model = EmergencyEventPredictor(
    embedding_module=embedding_module,  # Replace with actual CombinedEmbedding instance
    embed_dim=embed_dim,
    num_heads=num_heads,
    num_layers=num_layers,
    max_len=max_len
)

# Forward pass to get predictions
predictions = model(
    neighborhood_ids=neighborhood_ids,
    time_features=time_features,
    building_type_ids=building_type_ids,
    building_counts=building_counts,
    population=population,
    event_type_ids=event_type_ids,
    equipment_ids=equipment_ids
)

logger.info(f"Predictions Shape: {predictions.shape}")
logger.info(f"Predictions: {predictions}")

INFO:__main__:Predictions Shape: torch.Size([377])
INFO:__main__:Predictions: tensor([-8.4596e-01, -4.1118e-01, -1.0190e+00,  6.7146e-01, -9.5901e-02,
        -4.7638e-01,  1.0031e-01,  4.3799e-02,  6.2383e-01,  9.2592e-01,
         7.4286e-01,  4.9988e-01,  6.1780e-01,  3.0728e-01,  7.3109e-01,
         4.7078e-01,  7.5497e-01, -1.4704e-01,  5.6895e-01,  3.0706e-01,
         2.7444e-01,  1.4233e-03, -2.1236e-01, -1.0866e+00,  3.3767e-02,
        -5.1460e-02, -1.0146e+00, -2.4528e-01, -5.6907e-01, -1.0816e+00,
         7.5412e-02, -1.0771e+00, -3.3736e-01, -4.6823e-01, -4.4859e-01,
        -8.6387e-01, -1.8784e-01, -4.0200e-01, -1.7086e-01, -6.7623e-01,
        -4.1963e-01, -4.6192e-04, -3.8136e-01,  2.1020e-01, -5.4405e-01,
         3.2057e-01,  5.9276e-01, -3.2038e-01, -5.5142e-01, -2.3644e-01,
        -4.2400e-01, -8.8856e-02, -4.3567e-01, -2.8675e-02,  7.4900e-01,
        -5.0461e-02, -9.0832e-01, -3.6355e-01,  5.4279e-01,  3.1694e-01,
        -2.9457e-01, -6.3361e-01, -5.3953e-01,

: 

**Output Visulization**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Detach predictions and convert to NumPy for visualization
predictions_np = predictions.detach().numpy()

# Define the plot
plt.figure(figsize=(12, 100))
sns.heatmap(predictions_np, annot=True, cmap="coolwarm", cbar=True, fmt=".2f",
            xticklabels=["Day 1", "Day 2", "Day 3", "Day 4", "Day 5", "Day 6", "Day 7"],
            yticklabels=[f"Neighborhood {i+1}" for i in range(predictions_np.shape[0])])

# Add titles and labels
plt.title("Predicted Number of Events per Day for Each Neighborhood")
plt.xlabel("Day of the Week")
plt.ylabel("Neighborhood")
plt.show()

IndexError: Inconsistent shape between the condition and the input (got (377, 1) and (377,))

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Splitting neighborhoods into train and validation sets
train_indices, val_indices = train_test_split(
    np.arange(num_neighborhoods), test_size=0.2, random_state=42
)

# Prepare validation tensors
val_neighborhood_ids = neighborhood_ids[val_indices]
val_time_features = time_features[val_indices]
val_building_type_ids = building_type_ids[val_indices]
val_building_counts = building_counts[val_indices]
val_population = population[val_indices]
val_event_type_ids = event_type_ids[val_indices]
val_equipment_ids = equipment_ids[val_indices]
val_targets = targets[val_indices]

# Create Validation Dataset and DataLoader
val_dataset = NeighborhoodDataset(
    neighborhood_ids=val_neighborhood_ids,
    time_features=val_time_features,
    building_type_ids=val_building_type_ids,
    building_counts=val_building_counts,
    population=val_population,
    event_type_ids=val_event_type_ids,
    equipment_ids=val_equipment_ids,
    targets=val_targets
)

val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Validation Loop
def validate_model(model, val_dataloader):
    model.eval()  # Set model to evaluation mode
    all_predictions = []
    all_targets = []

    with torch.no_grad():  # Disable gradient computation
        i = 0
        for (_neighborhood_ids, _time_features, _building_type_ids, _building_counts,
             _population, _event_type_ids, _equipment_ids, _targets) in val_dataloader:
            
            # Forward pass
            _predictions = model(_neighborhood_ids, _time_features, _building_type_ids,
                                 _building_counts, _population, _event_type_ids, _equipment_ids)

            # Collect predictions and true values
            all_predictions.append(_predictions.cpu().numpy())
            all_targets.append(_targets.cpu().numpy())
    
    # Combine all batches
    all_predictions = np.concatenate(all_predictions, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)

    # Compute metrics
    print(f"all_targets {all_targets.shape}, all_predictions {all_predictions.shape}")
    mae = mean_absolute_error(all_targets, all_predictions)
    mse = mean_squared_error(all_targets, all_predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(all_targets, all_predictions)

    print(f"Validation Results:")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R² Score: {r2:.4f}")

    return all_predictions, all_targets

# Scatter Plot: Predicted vs. True Values
def plot_predictions_vs_true(all_predictions, all_targets):
    plt.figure(figsize=(8, 8))
    plt.scatter(all_targets, all_predictions, alpha=0.5, edgecolor='k')
    plt.plot([all_targets.min(), all_targets.max()], [all_targets.min(), all_targets.max()], 'r--')
    plt.title("Predicted vs True Event Counts")
    plt.xlabel("True Event Counts")
    plt.ylabel("Predicted Event Counts")
    plt.grid()
    plt.show()

# Heatmap: Neighborhood Predictions Over Time
def plot_heatmap(all_predictions, neighborhood_ids, days=["Day 1", "Day 2", "Day 3", "Day 4", "Day 5", "Day 6", "Day 7"]):
    plt.figure(figsize=(12, 20))
    sns.heatmap(all_predictions, annot=True, cmap="coolwarm", cbar=True, fmt=".2f",
                xticklabels=days,
                yticklabels=[f"Neighborhood {i}" for i in neighborhood_ids])
    plt.title("Predicted Event Counts per Neighborhood per Day")
    plt.xlabel("Days")
    plt.ylabel("Neighborhood")
    plt.show()

# Perform validation
all_predictions, all_targets = validate_model(model, val_dataloader)

# Visualize results
plot_predictions_vs_true(all_predictions, all_targets)
plot_heatmap(all_predictions, val_neighborhood_ids)
